## CNN sentence classifier - Round 2 of Hyperparameter tuning

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 16 ms, sys: 4.01 ms, total: 20 ms
Wall time: 19 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31.1 s, sys: 75 ms, total: 31.2 s
Wall time: 31.2 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:

# this is very quick, so we can use grid search on these models.

run_logfile = "runs2bal.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96.96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal",
                      model_dir        = "../models/CNNBal")
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 6s 8ms/step - loss: 0.6816 - accuracy: 0.6054 - val_loss: 0.5403 - val_accuracy: 0.7278
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154048/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.4797 - accuracy: 0.7703 - val_loss: 0.4162 - val_accuracy: 0.8142
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154048/assets
Epoch 3/20
288/288 [==============================] - 2s 5ms/step - loss: 0.3094 - accuracy: 0.8709 - val_loss: 0.2350 - val_accuracy: 0.9060
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154048/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1945 - accuracy: 0.9299 - val_loss: 0.2925 - val_accuracy: 0.8897
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1193 - accuracy: 0.9616 - val_

288/288 [==============================] - 2s 5ms/step - loss: 0.0493 - accuracy: 0.9847 - val_loss: 0.1892 - val_accuracy: 0.9507
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154202/assets
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0365 - accuracy: 0.9879 - val_loss: 0.2858 - val_accuracy: 0.9322
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0330 - accuracy: 0.9896 - val_loss: 0.3107 - val_accuracy: 0.9317
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0281 - accuracy: 0.9915 - val_loss: 0.3082 - val_accuracy: 0.9281
Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0256 - accuracy: 0.9932 - val_loss: 0.3726 - val_accuracy: 0.9304
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0212 - accuracy: 0.9926 - val_loss: 0.5702 - val_accuracy: 0.8897
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0207 -

Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0095 - accuracy: 0.9976 - val_loss: 0.4004 - val_accuracy: 0.9222
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0133 - accuracy: 0.9964 - val_loss: 0.4809 - val_accuracy: 0.9227
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0060 - accuracy: 0.9982 - val_loss: 0.3629 - val_accuracy: 0.9372
Epoch 20/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.3119 - val_accuracy: 0.9417



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6403 - accuracy: 0.6373 - val_loss: 0.4982 - val_accuracy: 0.7848
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154346/assets
Epoch 2/20
288/288 [=====

288/288 [==============================] - 2s 5ms/step - loss: 0.1266 - accuracy: 0.9552 - val_loss: 0.1588 - val_accuracy: 0.9471
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154455/assets
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0769 - accuracy: 0.9723 - val_loss: 0.2117 - val_accuracy: 0.9412
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0550 - accuracy: 0.9827 - val_loss: 0.1413 - val_accuracy: 0.9589
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-154455/assets
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0356 - accuracy: 0.9896 - val_loss: 0.4481 - val_accuracy: 0.9114
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0254 - accuracy: 0.9922 - val_loss: 0.3455 - val_accuracy: 0.9209
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0229 - accuracy: 0.9927 - val_loss: 0.2089 - val_accuracy: 0.9521
Epoch 10/20
2

Epoch 13/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0106 - accuracy: 0.9976 - val_loss: 0.3366 - val_accuracy: 0.9444
Epoch 14/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0130 - accuracy: 0.9957 - val_loss: 0.2346 - val_accuracy: 0.9580
Epoch 15/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0186 - accuracy: 0.9934 - val_loss: 0.5209 - val_accuracy: 0.9141
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0102 - accuracy: 0.9969 - val_loss: 0.7469 - val_accuracy: 0.8910
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0149 - accuracy: 0.9956 - val_loss: 0.4556 - val_accuracy: 0.9322
Epoch 18/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0070 - accuracy: 0.9978 - val_loss: 0.6520 - val_accuracy: 0.8987
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0123 - accuracy: 0.9957 - val_loss: 0.3295 - val_accuracy: 0.9494

...

Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0127 - accuracy: 0.9973 - val_loss: 0.3243 - val_accuracy: 0.9209
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0126 - accuracy: 0.9973 - val_loss: 0.3611 - val_accuracy: 0.9245
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0179 - accuracy: 0.9930 - val_loss: 0.4778 - val_accuracy: 0.9105
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0118 - accuracy: 0.9961 - val_loss: 0.3997 - val_accuracy: 0.9259
Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0085 - accuracy: 0.9969 - val_loss: 0.2650 - val_accuracy: 0.9507
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 0.8106 - val_accuracy: 0.8761
Epoch 13/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0194 - accuracy: 0.9935 - val_loss: 0.4647 - val_accuracy: 0.9195
Ep

288/288 [==============================] - 2s 6ms/step - loss: 0.0060 - accuracy: 0.9986 - val_loss: 0.3788 - val_accuracy: 0.9308
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0060 - accuracy: 0.9986 - val_loss: 0.4696 - val_accuracy: 0.9218
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.3302 - val_accuracy: 0.9467
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0080 - accuracy: 0.9979 - val_loss: 0.3882 - val_accuracy: 0.9367
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0090 - accuracy: 0.9972 - val_loss: 0.3884 - val_accuracy: 0.9394



##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 16
Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 3s

288/288 [==============================] - 2s 6ms/step - loss: 0.4122 - accuracy: 0.8144 - val_loss: 0.3311 - val_accuracy: 0.8608
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-161423/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.2464 - accuracy: 0.9005 - val_loss: 0.2160 - val_accuracy: 0.9168
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-161423/assets
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1456 - accuracy: 0.9453 - val_loss: 0.2551 - val_accuracy: 0.9060
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0802 - accuracy: 0.9721 - val_loss: 0.2768 - val_accuracy: 0.9155
Epoch 6/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0527 - accuracy: 0.9828 - val_loss: 0.1750 - val_accuracy: 0.9435
INFO:tensorflow:Assets written to: ../models/CNNBal/210404-161423/assets
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0409 -

288/288 [==============================] - 2s 6ms/step - loss: 0.0168 - accuracy: 0.9952 - val_loss: 0.3413 - val_accuracy: 0.9299
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0113 - accuracy: 0.9967 - val_loss: 0.4136 - val_accuracy: 0.9182
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0098 - accuracy: 0.9973 - val_loss: 0.2245 - val_accuracy: 0.9552
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0101 - accuracy: 0.9969 - val_loss: 0.4041 - val_accuracy: 0.9326
Epoch 14/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0091 - accuracy: 0.9977 - val_loss: 0.4607 - val_accuracy: 0.9241
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0085 - accuracy: 0.9970 - val_loss: 0.6504 - val_accuracy: 0.8942
Epoch 16/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0119 - accuracy: 0.9961 - val_loss: 0.5109 - val_accuracy: 0.9150
Epoch 17/20

KeyboardInterrupt: 





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/